In [1]:
!git clone https://github.com/Athe-kunal/Mod-HyDE.git

Cloning into 'Mod-HyDE'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 52 (delta 12), reused 46 (delta 6), pack-reused 0
Receiving objects: 100% (52/52), 9.48 MiB | 4.98 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [1]:
import os

os.chdir("Mod-HyDE")

In [2]:
with open("mod_hyde/data/all_book_data.txt","r") as f:
    all_books_data = f.read()

import json

youtube_transcripts_list = ["chunked_misc_transcripts.json","chunked_transcripts_undergrad.json","chunked_transcripts_mba.json"]
all_youtube_data = []
for file in youtube_transcripts_list:
    with open('mod_hyde/data/YouTube_API_Transcripts/'+file) as f:
        data = json.load(f)
    for id,data in data.items():
        for d in data:
            all_youtube_data.append(d['text'])

In [4]:
# !pip install -Uqq langchain
# !pip install -Uqq datasets
# !pip install accelerate -Uqq
# !pip install -q peft
# !pip install -q bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.0 MB/s eta 0:00:00


In [3]:
import re

all_books_data_list = re.split(r"(?<=[.?!])\s+", all_books_data)
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    separators = [". "],
    chunk_size=1750,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,

)

books_data_splitted_text = text_splitter.split_text(all_books_data)
all_data = books_data_splitted_text + all_youtube_data

## DISTILBERT

In [5]:
import torch; torch.version.cuda

'12.1'

In [1]:
# !pip install "unsloth[cu121] @ git+https://github.com/unslothai/unsloth.git"

In [4]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama", # Supports Llama, Mistral - replace this!
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True,
)

Unsloth: You passed in `unsloth/tinyllama` and `load_in_4bit = True`.
We shall load `unsloth/tinyllama-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = True, # @@@ IF YOU GET OUT OF MEMORY - set to True @@@
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


In [6]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name = "Qwen/Qwen1.5-1.8B"
# tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True)

In [6]:
import pandas as pd

df = pd.DataFrame(all_data,columns=["text"])

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [8]:
from datasets import Dataset
import pandas as pd

hf_dataset = Dataset.from_pandas(df)

In [9]:
hf_dataset

Dataset({
    features: ['text'],
    num_rows: 8394
})

In [10]:
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/8394 [00:00<?, ? examples/s]

In [11]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 8394
})

In [12]:
block_size = 1024
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)


Map (num_proc=4):   0%|          | 0/8394 [00:00<?, ? examples/s]

In [13]:
lm_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3516
})

In [14]:
tokenizer.decode(lm_datasets[0]['input_ids'])

'<s>Investment Fables Tall Tales about Stocks Fable: A brief fictitious story embodying a moral and using persons, animals or inanimate objects as characters; A falsehood or lie (WebsterÕs Dictionary)As investors, you have all been on the receiving end of sales pitches from brokers, friends and investment advisors about stocks that they claim will deliver spectacular returns. These stories not only sound persuasive and reasonable but are also backed up by evidence - anecdotal, in some cases, and statistical, in others - that the strategies work. When you try to implement them for your investments, though, you seldom can match their success on paper. All to often, you end up with buyer Õs remorse, poorer for the experience and promising yourselves that you will not fall for the allure of these stories again. All too often, you forget the lessons of past mistakes and are easy prey for the next big stock story.While there are literally hundreds of schemes to beat the market in circulation

In [15]:
tokenizer.decode(lm_datasets[0]['labels'])

'<s>Investment Fables Tall Tales about Stocks Fable: A brief fictitious story embodying a moral and using persons, animals or inanimate objects as characters; A falsehood or lie (WebsterÕs Dictionary)As investors, you have all been on the receiving end of sales pitches from brokers, friends and investment advisors about stocks that they claim will deliver spectacular returns. These stories not only sound persuasive and reasonable but are also backed up by evidence - anecdotal, in some cases, and statistical, in others - that the strategies work. When you try to implement them for your investments, though, you seldom can match their success on paper. All to often, you end up with buyer Õs remorse, poorer for the experience and promising yourselves that you will not fall for the allure of these stories again. All too often, you forget the lessons of past mistakes and are easy prey for the next big stock story.While there are literally hundreds of schemes to beat the market in circulation

In [16]:
# lm_datasets[0]['attention_mask']

In [17]:
# import peft
# from peft import LoraConfig, get_peft_model, PeftModel

# lora_config = LoraConfig(
#     r=4, #As bigger the R bigger the parameters to train.
#     lora_alpha=1, # a scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
#     target_modules=["query_key_value"], #You can obtain a list of target modules in the URL above.
#     lora_dropout=0.05, #Helps to avoid Overfitting.
#     bias="lora_only", # this specifies if the bias parameter should be trained.
#     task_type="CAUSAL_LM"
# )

In [19]:
from transformers import Trainer, TrainingArguments
model_name = "tl"
training_args = TrainingArguments(
    f"{model_name}-finetuned-ad",
    # evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    num_train_epochs = 1,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    lr_scheduler_type= 'cosine',
    seed = 42,
    gradient_accumulation_steps = 1
)


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets
)

In [21]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,516 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 1
\        /    Total batch size = 8 | Total steps = 440
 "-____-"     Number of trainable parameters = 6,307,840


Step,Training Loss


TrainOutput(global_step=440, training_loss=2.397261463512074, metrics={'train_runtime': 1538.8871, 'train_samples_per_second': 2.285, 'train_steps_per_second': 0.286, 'total_flos': 2.2484114888196096e+16, 'train_loss': 2.397261463512074, 'epoch': 1.0})

In [26]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model,tokenizer=tokenizer)
question = "What is the value of money?"
out = pipe(question,max_new_tokens=128,do_sample=True,min_new_tokens=10)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [29]:
question = "What is food?"
out = pipe(question,max_new_tokens=128,do_sample=True,min_new_tokens=10)
print(out[0]['generated_text'])

What is food? Food is what we eat when we live. Food consists of raw food or food processed according to the type of food consumed by the different populations that live in a particular area. Food is also made up of other items that we must consider and take into consideration as well during our production and consumption of food, namely items used to create energy in the form of carbohydrates, proteins, and fats. Some diets that emphasize foods, such as fast food chains, are based on convenience and do not require consumers to think about what is in their food, such as the fast food industry.


In [30]:
trainer.save_model("tiny_llama")

In [34]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [35]:
!zip -r tiny_llama.zip tiny_llama

from google.colab import files
files.download("tiny_llama.zip")

  adding: tiny_llama/ (stored 0%)
  adding: tiny_llama/adapter_config.json (deflated 54%)
  adding: tiny_llama/adapter_model.safetensors (deflated 7%)
  adding: tiny_llama/README.md (deflated 66%)
  adding: tiny_llama/training_args.bin (deflated 51%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>